In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw

import sys
sys.path.append('..')
from detector.input_pipeline import Pipeline

# Build a graph

In [ ]:
tf.reset_default_graph()

params = {
    'min_dimension': 768,
    'batch_size': 14,
    'image_height': 640,
    'image_width': 640,
}

pipeline = Pipeline(
    ['/home/dan/datasets/COCO/coco_person/train_shards/shard-0000.tfrecords'],
    is_training=True, params=params
)

dataset = pipeline.dataset
iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
init = iterator.make_initializer(dataset)
features, labels = iterator.get_next()
features.update(labels)

# Show an augmented image with boxes

In [ ]:
with open('coco_labels.txt', 'r') as f:
    label_encoder = {line.strip(): i for i, line in enumerate(f.readlines()) if line.strip()}

label_decoder = {i: n for n, i in label_encoder.items()}

In [ ]:
def draw_boxes(image, boxes, labels):
    image_copy = image.copy()
    draw = ImageDraw.Draw(image_copy, 'RGBA')

    for box, label in zip(boxes, labels):
        ymin, xmin, ymax, xmax = box

        fill = (255, 0, 0, 75)
        outline = 'black'

        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )
        draw.text((xmin + 2, ymin), label_decoder[label])

    return image_copy

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    output = sess.run(features)

In [ ]:
i = 0
image = Image.fromarray((255.0*output['images'][i]).astype('uint8'))
num_boxes = output['num_boxes'][i]
boxes = output['boxes'][i][:num_boxes].copy()
classes = output['labels'][i][:num_boxes].copy()

w, h = image.size
assert w % 128 == 0 and h % 128 == 0
scaler = np.array([h, w, h, w], dtype='float32')
boxes *= scaler

print(image.size)
draw_boxes(image, boxes, classes)